# Best Location for New Chinese Restaurant In Toronto


## Introduction/Business Problem
A customer is planning to open a new Chinese restaurant in Toronto. However, he is not sure where excatcly he should open the restaurant. He would prefer to open the restaurant at a lively place and also with a lot of diversities.

# Data
I'm going to scrape the location data of the neighbourhoods in Toronto from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M Then I'm goint to use the Foursquare location data to gest the venues information in all the neighbourhoods. Then I'm going to cluster the neighbourhoods according to their restaurants venues information. Then find the location that's close to a diversity of clusters of restaurant neighbourhoods.

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Scrape Neighbourhood Info from wikipedia.org

In [2]:
### Locate tabel in html and parse into dataframe
tabel = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=tabel[0]

In [3]:
### Fix the header
new_header=df.iloc[0]
df = df[1:]
df.columns = new_header

In [4]:
### Remove rows with unassigned Borough name
df = df[df.Borough != 'Not assigned']

In [7]:
### Merge rows with same Postcode
df=df.groupby('Postcode',as_index=False).agg(', '.join)
for i, row in df.iterrows():
    row['Borough']=row['Borough'].split(',')[0]

In [8]:
df.shape

(103, 3)

### Fetch and add the geolocation of the neighbourhoods in Toronto

In [9]:
! wget https://cocl.us/Geospatial_data -O Geospatial_data.csv

--2019-04-26 01:49:06--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-26 01:49:09--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-26 01:49:09--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-26 01:4

In [10]:
Geo_data = pd.read_csv('Geospatial_data.csv') 

In [11]:
Geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df=df.rename(columns={'Neighbourhood':'Neighborhood'})
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df['Latitude'] = None
df['Longitude'] = None
for i, row in df.iterrows():
    row['Latitude'] = Geo_data.loc[Geo_data['Postal Code'] == row['Postcode']]['Latitude'].values[0]
    row['Longitude'] = Geo_data.loc[Geo_data['Postal Code'] == row['Postcode']]['Longitude'].values[0]


In [14]:
#df['Latitude'] = df['Latitude'].map('{:,.6f}'.format)
#df['Longitude'] = df['Longitude'].map('{:,.6f}'.format)

In [15]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


### Create map of Toronto using latitude and longitude values

In [66]:
Latitude = 43.7054990607
Longitude = -79.3834667995
map_toronto = folium.Map(location=[43.7054990607, -79.3834667995], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(\
        [lat, lng],\
        radius=5,\
        popup=label,\
        color='blue',\
        fill=True,\
        fill_color='#3186cc',\
        fill_opacity=0.7,\
        parse_html=False).add_to(map_toronto)  
    
map_toronto      
    


In [18]:
#toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
#toronto_data.head()
toronto_data = df

### Get venues' info from FourSquare

In [29]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: B05ZHXKXG4IBJI3KRAPCZBURCMJGDCWHMYYGUL1J1K4LFB2L
CLIENT_SECRET:GXZHQLWBF4PEKRSPJHQUABJFJ4BKPRWW3AIS1OHDIDNAGHS1


In [30]:
toronto_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [31]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=B05ZHXKXG4IBJI3KRAPCZBURCMJGDCWHMYYGUL1J1K4LFB2L&client_secret=GXZHQLWBF4PEKRSPJHQUABJFJ4BKPRWW3AIS1OHDIDNAGHS1&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc264c7911fc4002c70869e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
# type your answer here

Toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

### Clean to get the restaurant venues' information

In [40]:

#Toronto_venues2
print(Toronto_venues.shape)
Toronto_venues=Toronto_venues[Toronto_venues['Venue Category'].str.contains('Restaurant')]                         
print(Toronto_venues.shape)
Toronto_venues.head()


(2250, 7)
(526, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
13,Woburn,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
14,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
15,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant


In [41]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",29,29,29,29,29,29
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",1,1,1,1,1,1
"Bathurst Manor, Downsview North, Wilson Heights",4,4,4,4,4,4
Bayview Village,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",12,12,12,12,12,12
Berczy Park,11,11,11,11,11,11
"Brockton, Exhibition Place, Parkdale Village",3,3,3,3,3,3
Business Reply Mail Processing Centre 969 Eastern,2,2,2,2,2,2


In [42]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 53 uniques categories.


## 3. Analyze Each Neighborhood

In [43]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,Cedarbrae,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,Cedarbrae,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
Toronto_onehot.shape

(526, 54)

In [45]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide, King, Richmond",0.000000,0.137931,0.00,0.068966,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.034483,0.034483,0.000000,0.034483,0.000000,0.034483,0.068966,0.00,0.000000,0.034483,0.00,0.034483,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.000000,0.103448,0.034483,0.00,0.103448,0.000000,0.00,0.137931,0.000000,0.0,0.034483,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,1.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
3,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.250000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
4,Bayview Village,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
5,"Bedford Park, Lawrence Manor East",0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.166667,0.00000,0.000000,0.00,0.000000,0.083333,0.000000,0.083333,0.000000,0.166667,0.083333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.083333,0.000000,0.00,0.083333,0.000000,0.0,0.000000,0.000000
6,Berczy Park,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000

In [46]:
Toronto_grouped.shape

(69, 54)

In [47]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0  American Restaurant  0.14
1      Thai Restaurant  0.14
2     Sushi Restaurant  0.10
3           Restaurant  0.10
4  Japanese Restaurant  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                   venue  freq
0       Asian Restaurant   1.0
1      Afghan Restaurant   0.0
2  Portuguese Restaurant   0.0
3    Japanese Restaurant   0.0
4      Jewish Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                     venue  freq
0     Fast Food Restaurant   1.0
1        Afghan Restaurant   0.0
2  New American Restaurant   0.0
3      Japanese Restaurant   0.0
4        Jewish Restaurant   0.0


----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0           Sushi Restaurant  0.25
1       Fast Food Restaurant  0.25
2                 Restaurant  0.25
3  Middle Eastern Restau

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Thai Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Asian Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
3,"Bathurst Manor, Downsview North, Wilson Heights",Fast Food Restaurant,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant
4,Bayview Village,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant


In [50]:
Toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide, King, Richmond",0.000000,0.137931,0.00,0.068966,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.034483,0.034483,0.000000,0.034483,0.000000,0.034483,0.068966,0.00,0.000000,0.034483,0.00,0.034483,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.000000,0.103448,0.034483,0.00,0.103448,0.000000,0.00,0.137931,0.000000,0.0,0.034483,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,1.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
3,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.250000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
4,Bayview Village,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000
5,"Bedford Park, Lawrence Manor East",0.000000,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.00000,0.00000,0.000,0.00,0.0,0.000000,0.000000,0.166667,0.00000,0.000000,0.00,0.000000,0.083333,0.000000,0.083333,0.000000,0.166667,0.083333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.083333,0.000000,0.00,0.083333,0.000000,0.0,0.000000,0.000000
6,Berczy Park,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000

## 4. Cluster Neighborhoods

In [51]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init= 'random',n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataf= 'random'rame
kmeans.labels_[0:20] 

array([2, 3, 0, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2], dtype=int32)

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944,0.0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887,2.0,Mexican Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
3,M1G,Scarborough,Woburn,43.771,-79.2169,2.0,Korean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,2.0,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395,NaN,NaN,NaN,NaN,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262,1.0,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846,0.0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395,2.0,American Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
### Drop Neighbourhoods where there aren't any restaurants open.
Toronto_merged = Toronto_merged.dropna()
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype('int32')
Toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887,2,Mexican Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
3,M1G,Scarborough,Woburn,43.771,-79.2169,2,Korean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,2,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262,1,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395,2,American Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.7574,-79.2733,2,Indian Restaurant,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Thai Restaurant
11,M1R,Scarborough,"Maryvale, Wexford",43.7501,-79.2958,2,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.7816,-79.3043,1,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant


In [74]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
7,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
17,North York,0,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant
25,North York,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
35,East York,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
74,York,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
76,West Toronto,0,Fast Food Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant
92,Etobicoke,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant
101,Etobicoke,0,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant


In [69]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Scarborough,1,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
13,Scarborough,1,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant
15,Scarborough,1,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,American Restaurant,Doner Restaurant
18,North York,1,Fast Food Restaurant,Asian Restaurant,Japanese Restaurant,Restaurant,American Restaurant
19,North York,1,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant
26,North York,1,Japanese Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant
27,North York,1,Asian Restaurant,Dim Sum Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant
46,Central Toronto,1,Fast Food Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Doner Restaurant
72,North York,1,Japanese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
85,Queen's Park,1,Japanese Restaurant,Portuguese Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant


In [70]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Scarborough,2,Mexican Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
3,Scarborough,2,Korean Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
4,Scarborough,2,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Vietnamese Restaurant,Doner Restaurant
8,Scarborough,2,American Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
10,Scarborough,2,Indian Restaurant,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Thai Restaurant
11,Scarborough,2,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
22,North York,2,Restaurant,Ramen Restaurant,Vietnamese Restaurant,Middle Eastern Restaurant,Fast Food Restaurant
28,North York,2,Fast Food Restaurant,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant
34,North York,2,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant
38,East York,2,Sushi Restaurant,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant


In [71]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Scarborough,3,Asian Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant
36,East York,3,Asian Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant


In [72]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
39,East York,4,Indian Restaurant,Hakka Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant


## Recommendation

#### From the map it's obvious that downtown toront is the most lively place in regards of number of neighbourhoods with restsrant open. And if the owner want to look for a place where there's more diversity, maybe he could consider nort to East York where different clusters of neighbourhoods meet.